<a href="https://colab.research.google.com/github/puneeshkhanna/Tensor-Parallelism/blob/master/tensor_parallelism_ffn_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import torch
import torch.nn as nn

In [ ]:
# Input of dimensions (batch size, no of words, embedding dimension of each word)
input = torch.randn(size=(1, 5, 10), dtype=torch.float32)

In [ ]:
embedding_dim = input.size(dim=2)
embedding_dim

10

## Back to back linear layers output

Tranformer architectures have back to back linear layers where the embedding dim
first goes from h to 4h and then back from 4h to h

```
(mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=False)          
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=False)
        )
```



In [ ]:
linear_h_to_4h = nn.Linear(in_features=embedding_dim, out_features=embedding_dim*4, bias=False)

# Input of shape [1,5,10] * W.T of shape [10, 40]
output = linear_h_to_4h(input)

# input of dimension (1,5,10) gets transformed to dimension (1,5,40)
output.shape

torch.Size([1, 5, 40])

In [ ]:
linear_4h_to_h = nn.Linear(in_features=embedding_dim*4, out_features=embedding_dim, bias=False)

# h_to_4h output of shape [1,5,40] * W.T of shape [40,10]
final_output = linear_4h_to_h(output)

# h_to_4h output of dimension (1,5,40) gets transformed back to dimension (1,5,10)
final_output.shape

torch.Size([1, 5, 10])

## Back to back linear layer outputs with Tensor parallelism

Assuming 2 devices, below code depicts how weights will be divided between the 2 devices.

Note that all the below code is executed on single device only with comments that which blocks will be executed on first device or second device.

In [ ]:
n_devices = 2
weight_parallel = int((embedding_dim*4)/n_devices)
weight_parallel

20

### First device - h_to_4h logic

In [ ]:
# First device weights of h_to_4h linear layer will be [out_features, in_feature] = [20, 10]
linear_h_to_4h_parallel = nn.Linear(in_features=embedding_dim, out_features=weight_parallel, bias=False)

# Set weights of this layer to the first 20 rows of the original weights
linear_h_to_4h_parallel.weight.data = linear_h_to_4h.weight[:weight_parallel,:]

# Input of shape [1,5,10] * W.T of shape [10, 20] ; note that in W.T - it is actually the columns which are divided; hence this is also known as column parallel linear
output_parallel_1 = linear_h_to_4h_parallel(input)

output_parallel_1.shape

torch.Size([1, 5, 20])

### Second device - h_to_4h logic

In [ ]:
# Second device weights of h_to_4h linear layer will be [out_features, in_feature] = [20, 10]
linear_h_to_4h_parallel = nn.Linear(in_features=embedding_dim, out_features=weight_parallel, bias=False)

# Set weights of this layer to the last 20 rows of the original weights
linear_h_to_4h_parallel.weight.data = linear_h_to_4h.weight[weight_parallel:,:]

# Input[1,5,10] * W.T[10, 20] ; note that in W.T - it is actually the columns which are divided; hence this is also known as column parallel linear
output_parallel_2 = linear_h_to_4h_parallel(input)

output_parallel_2.shape

torch.Size([1, 5, 20])

In [ ]:
# If we perform an all gather of output_parallel_1 and output_parallel_2, we will get the actual output of h_to_4h linear layer
# but since we have one more linear layer of 4h_to_h, we can continue with the tensor parallel outputs.

### First device - 4h_to_h logic

In [ ]:
# First device weights of 4h_to_h linear layer will be [out_features, in_features] = [10, 20]
linear_4h_to_h_parallel = nn.Linear(in_features=weight_parallel, out_features=embedding_dim, bias=False)

# Set weights of this layer to the first 20 columns of the original weights
linear_4h_to_h_parallel.weight.data = linear_4h_to_h.weight[:,:weight_parallel]

# output_parallel_1 of shape [1,5,20] * W.T of shape [20,10] ; note that in W.T - it is actually the rows which are divided; hence this is also known as row parallel linear
final_output_parallel_1 = linear_4h_to_h_parallel(output_parallel_1)

### Second device - 4h_to_h logic

In [ ]:
# Second device weights of 4h_to_h linear layer will be [out_features, in_features] = [10, 20]
linear_4h_to_h_parallel = nn.Linear(in_features=weight_parallel, out_features=embedding_dim, bias=False)

# Set weights of this layer to the last 20 columns of the original weights
linear_4h_to_h_parallel.weight.data = linear_4h_to_h.weight[:,weight_parallel:]

# output_parallel_2 of shape [1,5,20] * W.T of shape [20,10] ; note that in W.T - it is actually the rows which are divided; hence this is also known as row parallel linear
final_output_parallel_2 = linear_4h_to_h_parallel(output_parallel_2)

### Add tensor parallel outputs of 4h_to_h using torch.add; it will be an all reduce operation when using actual 2 devices

In [ ]:
# In actual tensor parallelism between 2 devices, this will be all reduce operation
final_output_tp = torch.add(final_output_parallel_1, final_output_parallel_2)
final_output_tp.shape

torch.Size([1, 5, 10])

In [ ]:
final_output_tp.shape

torch.Size([1, 5, 10])

## Compare final_output and final_output_tp using allclose

In [ ]:
print(torch.allclose(final_output, final_output_tp, rtol=1e-05, atol=1e-05))

True


In [ ]:
final_output

tensor([[[-0.5751,  0.4617,  0.1493,  0.0056,  0.2218, -0.1540,  0.1138,
          -0.3654,  0.7666, -0.3740],
         [-0.3632,  0.2662,  0.1769, -0.1032,  0.0542, -0.2100,  0.2093,
          -0.0711,  0.5230, -0.5673],
         [ 0.0499,  0.1617, -0.1641, -0.1193,  0.0013,  0.0018, -0.0664,
           0.0953,  0.0862, -0.1954],
         [ 0.0680,  0.3866, -0.1191, -0.0693, -0.6244, -0.1078,  0.2118,
           0.0076, -0.0895, -0.2661],
         [ 0.3321, -0.2314,  0.3488, -0.1860,  0.3012, -0.4169, -0.4020,
           0.0484, -0.1718,  0.3321]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
final_output_tp

tensor([[[-0.5751,  0.4617,  0.1493,  0.0056,  0.2218, -0.1540,  0.1138,
          -0.3654,  0.7666, -0.3740],
         [-0.3632,  0.2662,  0.1769, -0.1032,  0.0542, -0.2100,  0.2093,
          -0.0711,  0.5230, -0.5673],
         [ 0.0499,  0.1617, -0.1641, -0.1193,  0.0013,  0.0018, -0.0664,
           0.0953,  0.0862, -0.1954],
         [ 0.0680,  0.3866, -0.1191, -0.0693, -0.6244, -0.1078,  0.2118,
           0.0076, -0.0895, -0.2661],
         [ 0.3321, -0.2314,  0.3488, -0.1860,  0.3012, -0.4169, -0.4020,
           0.0484, -0.1718,  0.3321]]], grad_fn=<AddBackward0>)